In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Underweight"
cohort = "GSE57802"

# Input paths
in_trait_dir = "../../input/GEO/Underweight"
in_cohort_dir = "../../input/GEO/Underweight/GSE57802"

# Output paths
out_data_file = "../../output/preprocess/Underweight/GSE57802.csv"
out_gene_data_file = "../../output/preprocess/Underweight/gene_data/GSE57802.csv"
out_clinical_data_file = "../../output/preprocess/Underweight/clinical_data/GSE57802.csv"
json_path = "../../output/preprocess/Underweight/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE57802_family.soft.gz', 'GSE57802_series_matrix.txt.gz']
Identified SOFT files: ['GSE57802_family.soft.gz']
Identified matrix files: ['GSE57802_series_matrix.txt.gz']



Background Information:
!Series_title	"Transcriptome Profiling of patients with 16p11.2 rearrangements"
!Series_summary	"The 600kb BP4-BP5 16p11.2 CNV (copy number variant) is associated with neuroanatomical, neurocognitive and metabolic disorders.  These recurrent rearrangements are associated with reciprocal phenotypes such as obesity and underweight, macro- and microcephaly, as well as autism spectrum disorder (ASD) and schizophrenia. Here we interrogated the transcriptome of individuals carrying reciprocal CNVs in 16p11.2."
!Series_summary	"The genome-wide transcript perturbations correlated with clinical endophenotypes of the CNV and were enriched for genes associated with ASD. We uncovered a significant correlation between copy number changes and expression levels of genes mutated in ciliopathies."
!Series_overall_design	"Transcriptome profiles of lymphoblastoid cell lines of 50 16p11.2 deletion carriers, 31 16p11.2 duplication carriers and 17 controls."

Sample Characteristics 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptome profiles
# which suggests gene expression data is available
is_gene_available = True

# 2. Clinical Feature Extraction

# 2.1 Data Availability
# For trait (Underweight):
# From background info, we see that the dataset includes data on 16p11.2 CNV which is associated with
# underweight/obesity. The key 3 'copy number 16p11.2' or key 4 'genotype' can be used to determine underweight.
# We'll use key 4 as it's more specific and differentiates between deletion (del) and duplication (dup)
trait_row = 4

# For age:
# Age information is available under key 2
age_row = 2

# For gender:
# Gender information is available under key 1
gender_row = 1

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert genotype information to binary trait (Underweight)."""
    if not value or 'genotype:' not in value:
        return None
    # Extract value after the colon and strip whitespace
    genotype = value.split('genotype:')[1].strip()
    
    # Based on background information, 16p11.2 duplication is associated with underweight
    # and deletion with obesity. So duplication = 1 (presence of underweight trait)
    if genotype == '600kbdup':
        return 1
    elif genotype == '600kbdel' or genotype == 'Control':
        return 0
    else:
        return None

def convert_age(value):
    """Convert age information to continuous numeric values."""
    if not value or 'age:' not in value:
        return None
    # Extract value after the colon and strip whitespace
    age_str = value.split('age:')[1].strip()
    if age_str == 'NA':
        return None
    try:
        # Convert to float to handle decimal ages
        return float(age_str)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender information to binary (0=female, 1=male)."""
    if not value or 'gender:' not in value:
        return None
    # Extract value after the colon and strip whitespace
    gender = value.split('gender:')[1].strip()
    if gender == 'F':
        return 0
    elif gender == 'M':
        return 1
    else:
        return None

# 3. Save Metadata - Conduct initial filtering
# Check if trait data is available
is_trait_available = trait_row is not None
# Save the information using the library function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# For now, just define the variables and functions for later use
# since we don't have access to the clinical data yet
# The actual extraction will be done in a subsequent step
print(f"Trait row: {trait_row}")
print(f"Age row: {age_row}")
print(f"Gender row: {gender_row}")
print("Conversion functions defined for trait, age, and gender.")
print(f"is_gene_available: {is_gene_available}")
print(f"is_trait_available: {is_trait_available}")


Trait row: 4
Age row: 2
Gender row: 1
Conversion functions defined for trait, age, and gender.
is_gene_available: True
is_trait_available: True


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')

Gene expression data shape: (54715, 99)


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (1007_PM_s_at, 1053_PM_at, etc.) are Affymetrix probe IDs
# from the Affymetrix GeneChip Human Genome U133 Plus 2.0 Array
# They are not human gene symbols and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in the gene annotation dataframe that store gene identifiers and gene symbols
# From the preview, we can see that 'ID' contains the probe identifiers matching those in the gene expression data
# 'Gene Symbol' contains the corresponding gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get a gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of the gene mapping dataframe:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few rows of the gene expression data after mapping:")
print(gene_data.head())

# Save the gene expression data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (41796, 2)
First few rows of the gene mapping dataframe:
             ID    Gene
0  1007_PM_s_at    DDR1
1    1053_PM_at    RFC2
2     117_PM_at   HSPA6
3     121_PM_at    PAX8
4  1255_PM_g_at  GUCA1A
Gene expression data shape after mapping: (18989, 99)
First few rows of the gene expression data after mapping:
       GSM1389621  GSM1389622  GSM1389623  GSM1389624  GSM1389625  GSM1389626  \
Gene                                                                            
A1BG       5.1530      4.2584      4.5459      4.9238      5.1245      5.0254   
A1CF       3.1398      3.1938      3.2668      3.3192      3.1315      2.8158   
A2BP1     11.9172     12.2129     11.5026     11.8598     12.2363     12.2537   
A2LD1      8.1127      7.3163      7.9812      7.6193      8.3075      8.1315   
A2M        6.0117      4.9206      5.4258      5.1350      5.4096      5.1066   

       GSM1389627  GSM1389628  GSM1389629  GSM1389630  ...  GSM1389710  \
Gene           

Gene expression data saved to ../../output/preprocess/Underweight/gene_data/GSE57802.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print(f"First few gene symbols after normalization: {list(gene_data.index[:10])}")

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the functions defined in Step 2
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical features extracted:")
print(preview_df(clinical_features))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview:")
print(preview_df(linked_data))

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_trait_biased}")

# 6. Conduct quality check and save the cohort information
note = "This dataset contains data on 16p11.2 duplication carriers associated with underweight."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (18622, 99)
First few gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Underweight/gene_data/GSE57802.csv
Clinical features extracted:
{'GSM1389621': [0.0, 46.0, 1.0], 'GSM1389622': [0.0, 33.0, 0.0], 'GSM1389623': [0.0, nan, 1.0], 'GSM1389624': [0.0, nan, 0.0], 'GSM1389625': [0.0, 22.0, 1.0], 'GSM1389626': [0.0, 52.0, 1.0], 'GSM1389627': [0.0, 25.0, 1.0], 'GSM1389628': [0.0, 31.0, 0.0], 'GSM1389629': [0.0, 60.0, 1.0], 'GSM1389630': [0.0, nan, 1.0], 'GSM1389631': [0.0, 40.0, 1.0], 'GSM1389632': [0.0, 50.0, 1.0], 'GSM1389633': [0.0, 51.0, 1.0], 'GSM1389634': [0.0, 39.0, 1.0], 'GSM1389635': [0.0, 6.0, 1.0], 'GSM1389636': [0.0, 51.0, 1.0], 'GSM1389637': [0.0, 56.0, 0.0], 'GSM1389638': [0.0, 16.0, 1.0], 'GSM1389639': [0.0, 41.0, 1.0], 'GSM1389640': [0.0, 31.0, 0.0], 'GSM1389641': [0.0, 35.0, 1.0], 'GSM1389642': [0.0, 4.0, 1.0], 'GSM1389643': [0.0, 10.0, 0.0], 'GSM1389644': [0.0, 12.0, 0.0], 'GSM1389645': [0.0, 7.0, 1.0], 'GSM1389646': [0.0, 6.0, 1.0], 'GSM1389647': [0.0, 1.4, 1.0], 'GSM1389648': [0.0, 10.0,

{'Underweight': [0.0, 0.0, 0.0, 0.0, 0.0], 'Age': [46.0, 33.0, nan, nan, 22.0], 'Gender': [1.0, 0.0, 1.0, 0.0, 1.0], 'A1BG': [5.153, 4.2584, 4.5459, 4.9238, 5.1245], 'A1BG-AS1': [4.4607, 4.087, 4.518, 4.7735, 4.4022], 'A1CF': [3.1398, 3.1938, 3.2668, 3.3192, 3.1315], 'A2M': [6.011699999999999, 4.9206, 5.425800000000001, 5.135, 5.409599999999999], 'A2ML1': [5.9201999999999995, 5.6129999999999995, 5.434799999999999, 5.4277999999999995, 5.3739], 'A4GALT': [3.5943, 3.7949, 3.6152, 3.1873, 3.7345], 'A4GNT': [3.1132, 3.0257, 3.019, 3.1873, 3.133], 'AAA1': [5.8944, 6.0103, 5.616, 5.5187, 5.4543], 'AAAS': [4.5034, 4.3817, 4.6228, 4.5142, 4.6848], 'AACS': [8.0014, 8.192, 7.9628, 7.7601, 8.2706], 'AACSP1': [2.4715, 2.7737, 2.7885, 3.0716, 2.7716], 'AADAC': [2.8244, 2.771, 2.9491, 2.6525, 2.9194], 'AADACL2': [2.6007, 2.6733, 2.6212, 2.5502, 2.4922], 'AADAT': [3.3793, 2.9596, 2.9957, 3.0025, 2.8109], 'AAGAB': [16.0607, 16.642, 17.156200000000002, 15.819600000000001, 16.7413], 'AAK1': [29.1696, 31.

Linked data shape after handling missing values: (99, 18625)
For the feature 'Underweight', the least common label is '1.0' with 32 occurrences. This represents 32.32% of the dataset.
The distribution of the feature 'Underweight' in this dataset is fine.

Quartiles for 'Age':
  25%: 6.4
  50% (Median): 14.7
  75%: 36.5
Min: 0.6
Max: 76.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 41 occurrences. This represents 41.41% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Is trait biased: False
Data quality check result: Usable


Linked data saved to ../../output/preprocess/Underweight/GSE57802.csv
